<a href="https://colab.research.google.com/github/mohnfeld/Data-Analytics-Batch-Fall-23/blob/main/Copy_of_Transformations_%26_Joins_in_Pandas_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# this is a hack to allow displaying more than one result per notebook cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
cars = pd.read_csv("https://raw.githubusercontent.com/juliandnl/redi_ss20/master/cars.csv")

# Part 1 - Transformations

The following exercises are about doing row-wise transformations of string columns. [Docs](https://pandas.pydata.org/docs/user_guide/text.html#) on working with string data in pandas. [Built-in functions](https://pandas.pydata.org/docs/reference/series.html#string-handling) on string columns. Note that the exercises cover different implementation variations.

As a rule of thumb it is better to use dataframe built-in functionality if available for the logic you want to implement. Lower-level abstractions like `apply` can help if the functionality is not available.

## Exercise 1
Create a new column `has_power` that indicates (with booleans) if the value in the `Variant` column contains the word "Power"

1. Use `apply` with a `def` function and anonymous `lambda` function
2. Use only dataframe built-in functionality



In [ ]:
## def function

def has_power_or_no(row):
  if "Power" in row["Variant"]:
    return True
  else:
    return False
cars["has_power"] = cars.apply(has_power_or_no, axis=1)

## lambda
cars.apply(lambda row: True if "Power" in row['Variant']  else False, axis=1)

##built in
cars["has_power"]= np.where(cars["Variant"].str.contains("Power"),True,False)


cars["has_power"].value_counts()


0       False
1       False
2       False
3       False
4        True
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

False    9491
True      509
Name: has_power, dtype: int64

## Exercise 2
Create an abbreviation of the `Make` of a car. Create a new column `abbrev_brand` that contains the first 3 letters in uppercase of the value from the `Make` columns. For example, if `Make` is `Chrysler`, `abbrev_brand` should be `CHR`.

1. Use `apply` with a regular `def` function and anonymous `lambda` function
2. Use only dataframe built-in functionality

In [ ]:
##def function

def make_cap_abb(capital_abb):
  return capital_abb[0:3].upper()
cars["abbrev_brand"]=cars["Make"].apply(make_cap_abb)

##lambda

cars["abbrev_brand"]=cars["Make"].apply(lambda capital_abb: capital_abb[0:3].upper())

##built in
cars["abbrev_brand"]=cars["Make"].str[0:3].str.upper()
cars

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,has_power,abbrev_brand
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,False,VOL
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,False,FOR
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,False,VOL
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,False,FOR
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,True,VOL
...,...,...,...,...,...,...,...,...,...,...,...
9995,Ford,Focus III,2016,2.0 Se,67000,399000,5.0,Hatchback,Bs.as. G.b.a. Oeste,False,FOR
9996,Volkswagen,Bora,2012,1.9 Trendline I 100cv,120000,240000,4.0,Sedán,Buenos Aires Interior,False,VOL
9997,Ford,Fiesta Kinetic Design,2012,1.6 Design 120cv Titanium,89000,250000,5.0,Hatchback,Tucumán,False,FOR
9998,Ford,Fiesta Kinetic Design,2013,1.6 Design 120cv Titanium,76000,295000,5.0,Hatchback,Buenos Aires Interior,False,FOR


# Part 2 - Joins

The following exercises cover different join types. Check the [merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) docs for help on the `how` parameter.

We create the same dataframe as seen in the class to use for the joins. It is computed based on 'cars' and has following properties
- Average price **in euros €** of car models per brand (brand = Make). Columns: `Make, Model, avg_price`
- 3 additional "invented" rows
- All `Mercedes Benz` cars are removed

In [3]:
Price_Euro = cars['Price'] * 0.0027
price_column_position = cars.columns.get_loc('Price')
position = price_column_position + 1
cars.insert(position, 'Price_Euro', Price_Euro)

avg_model_prices = cars.groupby(['Make', 'Model'])['Price_Euro'].mean().reset_index()
avg_model_prices = avg_model_prices.rename(columns={'Price_Euro': 'avg_price'})
avg_model_prices = avg_model_prices.loc[avg_model_prices.loc[:, 'Make'] != 'Mercedes Benz', :]

invented_rows = pd.DataFrame(
    data = [('Ford', 'Lo', 158703.340), ('Ford', 'Hi', 324235.670), ('Ford', 'Cheap', 6533.700)],
    columns=['Make', 'Model', 'avg_price']
)

avg_model_prices = pd.concat([invented_rows, avg_model_prices], axis=0, ignore_index=True)

## Exercise 1
Do a RIGHT OUTER JOIN on the dataset and compare the result to the LEFT OUTER JOIN we've done during the class.

1. How do the `Mercedes Benz` rows compare?
2. How do the "invented rows" from `avg_model_prices` compare?
3. What's the shape of the result? Explain what you think it should be and then check it.
4. How can you achieve the behaviour of the LEFT OUTER JOIN from the class, but doing the RIGHT OUTER JOIN (`merge` with `how='right' parameter`)?

In [ ]:
avg_model_prices


In [7]:
right_join = pd.merge(cars, avg_model_prices, on=['Make', 'Model'], how='right')
right_join.head()

### q1: when we run these codes we see that for the right join, we don't have any mercedes benz data as we dropped that in the right table, therefore the joined data
#does not have it either, but for the left join mercedes benz misses the avg price values as the left table does not have such a column and the joined does not have it either
right_join.loc[right_join["Make"]=="Mercedes Benz"]
left_join.loc[left_join["Make"]=="Mercedes Benz"]

### q2:


,Make,Model,Year,Variant,Kms,Price,Price_Euro,Doors,Kind,Location,avg_price
0,Ford,Lo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158703.340000
1,Ford,Hi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324235.670000
2,Ford,Cheap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6533.700000
3,Chrysler,PT Cruiser,2011.0,2.4 Touring,70000.0,250000.0,675.0,5.0,Hatchback,Capital Federal,524.525625
4,Chrysler,PT Cruiser,2011.0,2.4 Touring,43000.0,225000.0,607.5,5.0,Hatchback,Bs.as. G.b.a. Norte,524.525625


In [10]:
left_join = pd.merge(cars, avg_model_prices, on=['Make', 'Model'], how='left')
left_join.head()

,Make,Model,Year,Variant,Kms,Price,Price_Euro,Doors,Kind,Location,avg_price
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,972.000,4.0,Sedán,Córdoba,1042.829090
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,864.000,2.0,Pick-Up,Entre Ríos,1548.109398
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,566.946,5.0,Hatchback,Bs.as. G.b.a. Sur,643.159240
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,2154.600,4.0,Pick-Up,Neuquén,1548.109398
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,394.200,3.0,Hatchback,Córdoba,405.814436


## Exercise 2
Do a FULL OUTER JOIN and INNER JOIN on the datasets and answer following questions

  1. Check the shape of the result and compare it to the shape of cars/avg_model_prices.
    * What do you observe and how can you explain it?
  2. Check how `Mercedes Benz` rows look in the result and explain why
  3. Check how the "invented rows" from `avg_model_prices` look in the result and explain why

## Exercise 3
Compute the price difference (based on the euros price) of each car, compared to the average price of each brand (`Make`). Do a join as part of the solution. Which type of join do you use? Explain your choice.

# Bonus Dataset: IMDB movie dataset
We'll look at the movie datasets provided by [IMDb](https://www.imdb.com/). The datasets are described [here](https://developer.imdb.com/non-commercial-datasets/). We'll work with a sample of the `title.basics` dataset (which includes only the top 10,000 most-voted movies) and `title.ratings` datasets.


In [ ]:
movies = pd.read_csv("https://raw.githubusercontent.com/obreit/redi/master/imdb_data/top_voted.tsv", sep='\t', header=0)

## Exercise 1
The goal is to get long-running movies (i.e. with `runtimeMinutes` more than 2 hours). Notice that the obvious expression `movies['runtimeMinutes'] > 120` crashes. Try it yourself and try to explain why this is not working (for a hint, see the [details](https://developer.imdb.com/non-commercial-datasets/#imdb-dataset-details) section of the imdb page). Before going to the next hints, try to implement a solution to this problem by yourself.

The `runtimeMinutes` column isn't of numeric type, because it contains `\N` values that can't be interpreted as numbers. So we need to somehow update those values and make a numeric column out of it.
1. Replace all rows where `runtimeMinutes` contains the imdb encoding for a missing value (`\N`) with `pd.NA` (the pandas type which represents a missing/null value). Note that you might run into some syntax issues. Try to find out how to overcome those.
2. Try to cast the column to a numeric type like this: `.astype(int)`. Why doesn't it work?
3. Try to find alternative ways to make the casting work.
4. Filter the movies that are longer than 2 hours

## Exercise 2
Count how many movies there are per genre. Notice that the `genres` column contains the information about the genre of a movie. But a movie can be assigned to multiple genres. We want to count the movie for every genre.

For example, if we have two movies `sad movie with genres: drama, action` and `dramedy with genres: drama, comedy` then the result counts should be `action: 1, comedy: 1, drama: 2`.

However, the `genres` column is not in a format that makes this counting easy (it's simply a string column). You'll need to transform the `genres` column in a way to simplify the counting. Part of this transformation will be to use the [explode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) function.   

## Exercise 3
Find out some information about the movie ratings. Specifically, answer following questions
1. What's the movie with the highest average rating?
2. What's the movie with the lowest average rating among the movies with at least 1 million votes (if there are multiple with the same rating, return all of them)
3. Look at the last 10 years and count the number of movies per year as well as the average rating per year

In order to answer the questions, you need to combine the `ratings` dataset with the movies dataset.

In [ ]:
ratings = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep='\t', header=0)